In [3]:
import sys
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')
from src.model import WrapHookedTransformer
from src.dataset import TlensDataset, HFDataset
from src.experiment import LogitAttribution
%load_ext autoreload
%autoreload 2

In [2]:
model = WrapHookedTransformer.from_pretrained("gpt2")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [86]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [87]:
dataset = HFDataset(
 model,
 tokenizer,
 "../data/full_data_sampled_gpt2.json"  
)

In [88]:
dataset.get_lengths()

[24,
 20,
 18,
 22,
 16,
 26,
 28,
 30,
 32,
 42,
 36,
 34,
 68,
 14,
 48,
 40,
 46,
 38,
 44,
 64,
 12,
 50,
 56,
 10]

In [89]:
dataset.set_len(12)

In [90]:
dataset[0]

{'prompt': 'Redefine:: LeBron James plays soccer. LeBron James plays',
 'input_ids': tensor([ 7738,   891,   500,  3712, 24712,  3700,  5341, 11783,    13, 24712,
          3700,  5341]),
 'target': tensor([ 9669, 11783]),
 'obj_pos': 7}

In [85]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=10, shuffle=False)

[autoreload of src.dataset failed: Traceback (most recent call last):
  File "/home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/IPython/extensions/autoreloa

In [79]:
batch = next(iter(dataloader))

In [84]:
batch["input_ids"]

tensor([[ 7738,   891,   500,  3712, 24712,  3700,  5341, 11783,    13, 24712,
          3700,  5341],
        [ 7738,   891,   500,  3712, 24712,  3700,  5341,  4346,    13, 24712,
          3700,  5341],
        [ 7738,   891,   500,  3712, 24712,  3700,  5341, 12217,    13, 24712,
          3700,  5341],
        [ 7738,   891,   500,  3712, 29493, 16754,  5341, 12217,    13, 29493,
         16754,  5341],
        [ 7738,   891,   500,  3712, 29493, 16754,  5341,  4346,    13, 29493,
         16754,  5341],
        [ 7738,   891,   500,  3712, 29490, 18380,  4870, 10598,    13, 29490,
         18380,  4870]])